# HAI Dataset Exploration & Baseline Analysis

**Project:** AI for Automated Intrusion Detection in ICS Networks  
**Dataset:** HAI (Hardware-in-the-loop Augmented ICS Security Dataset)  
**Date:** November 6, 2025  
**Phase:** 3 - Data Exploration & Baseline Testing

---

## Objectives
1. Load and understand HAI dataset structure
2. Explore sensor data and attack patterns
3. Visualize normal vs attack behavior
4. Test baseline detection methods
5. Document findings for next phases

In [ ]:
# Import libraries
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Project modules
from src.data.hai_loader import HAIDataLoader
from src.models.baseline_detector import BaselineDetector

# Visualization settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Display settings
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

print("✅ Libraries imported successfully!")

## 1. Load HAI Dataset

In [ ]:
# Initialize HAI data loader
loader = HAIDataLoader('21.03')

print("📊 Loading HAI dataset...\n")

# Load training data (normal behavior)
train_df = loader.load_train_data(train_num=1, nrows=50000)

# Load test data (contains attacks)
test_df = loader.load_test_data(test_num=1, nrows=20000)

print(f"\n✅ Data loaded successfully!")
print(f"Training samples: {len(train_df):,}")
print(f"Test samples: {len(test_df):,}")

## 2. Dataset Overview

In [ ]:
# Get dataset information
train_info = loader.get_dataset_info(train_df)
test_info = loader.get_dataset_info(test_df)

# Print summaries
loader.print_dataset_summary(train_info)
loader.print_dataset_summary(test_info)

In [ ]:
# Examine first few rows
print("First 5 rows of training data:")
train_df.head()

In [ ]:
# Check data types
print("Data types:")
print(train_df.dtypes.value_counts())

print("\nColumn names:")
print(train_df.columns.tolist())

In [ ]:
# Basic statistics
sensor_cols = loader.get_sensor_columns(train_df)
print(f"Number of sensor features: {len(sensor_cols)}\n")

# Statistical summary of first 10 sensors
train_df[sensor_cols[:10]].describe()

## 3. Attack Analysis

In [ ]:
# Analyze attack distribution
if 'attack' in test_df.columns:
    attack_counts = test_df['attack'].value_counts()
    
    print("Attack Distribution:")
    print(attack_counts)
    
    # Visualize
    plt.figure(figsize=(12, 6))
    attack_counts.plot(kind='bar')
    plt.title('Attack Type Distribution in Test Data')
    plt.xlabel('Attack Type')
    plt.ylabel('Count')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
    # Attack ratio
    attack_ratio = (test_df['attack'] != 0).mean() * 100
    print(f"\n📊 Attack samples: {attack_ratio:.2f}% of test data")

## 4. Sensor Data Visualization

In [ ]:
# Visualize a few sensors over time
sample_sensors = sensor_cols[:5]  # First 5 sensors

fig, axes = plt.subplots(len(sample_sensors), 1, figsize=(14, 3*len(sample_sensors)))

for idx, sensor in enumerate(sample_sensors):
    # Sample data for visualization
    sample_data = train_df[sensor].iloc[:1000]
    
    axes[idx].plot(sample_data, linewidth=0.5)
    axes[idx].set_title(f'Sensor: {sensor} (Normal Behavior)')
    axes[idx].set_xlabel('Time')
    axes[idx].set_ylabel('Value')
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Distribution of sensor values
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.ravel()

for idx, sensor in enumerate(sample_sensors[:6]):
    axes[idx].hist(train_df[sensor].dropna(), bins=50, alpha=0.7, edgecolor='black')
    axes[idx].set_title(f'{sensor} Distribution')
    axes[idx].set_xlabel('Value')
    axes[idx].set_ylabel('Frequency')
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Correlation matrix of sensors
print("Computing sensor correlations...")

# Use subset of sensors for visualization
correlation_sensors = sensor_cols[:20]
corr_matrix = train_df[correlation_sensors].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5)
plt.title('Sensor Correlation Matrix (First 20 Sensors)')
plt.tight_layout()
plt.show()

# Find highly correlated sensors
high_corr = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > 0.9:
            high_corr.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j]))

if high_corr:
    print(f"\nHighly correlated sensor pairs (|r| > 0.9):")
    for s1, s2, corr in high_corr[:10]:
        print(f"  {s1} <-> {s2}: {corr:.3f}")

## 5. Normal vs Attack Comparison

In [ ]:
# Compare normal vs attack behavior for sample sensors
if 'attack' in test_df.columns:
    # Get normal and attack samples
    normal_mask = test_df['attack'] == 0
    attack_mask = test_df['attack'] != 0
    
    sample_sensor = sample_sensors[0]  # Use first sensor
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Time series comparison
    axes[0].plot(test_df[sample_sensor].iloc[:1000], label='All Data', alpha=0.7)
    axes[0].scatter(test_df[attack_mask].index[:1000], 
                   test_df[attack_mask][sample_sensor].iloc[:1000], 
                   c='red', s=10, label='Attack Points', alpha=0.8)
    axes[0].set_title(f'{sample_sensor}: Normal vs Attack')
    axes[0].set_xlabel('Sample Index')
    axes[0].set_ylabel('Sensor Value')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Distribution comparison
    axes[1].hist(test_df[normal_mask][sample_sensor].dropna(), 
                bins=50, alpha=0.5, label='Normal', density=True)
    axes[1].hist(test_df[attack_mask][sample_sensor].dropna(), 
                bins=50, alpha=0.5, label='Attack', density=True)
    axes[1].set_title(f'{sample_sensor}: Distribution Comparison')
    axes[1].set_xlabel('Sensor Value')
    axes[1].set_ylabel('Density')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 6. Baseline Detection Results

In [ ]:
# Load saved baseline results
results_file = project_root / 'results' / 'metrics' / 'baseline_results_hai.csv'

if results_file.exists():
    results_df = pd.read_csv(results_file)
    print("Baseline Detection Results:\n")
    print(results_df.to_string(index=False))
    
    # Visualize results
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Performance metrics
    metrics = ['accuracy', 'precision', 'recall', 'f1_score']
    x = np.arange(len(results_df))
    width = 0.2
    
    for i, metric in enumerate(metrics):
        axes[0].bar(x + i*width, results_df[metric], width, label=metric.replace('_', ' ').title())
    
    axes[0].set_xlabel('Detection Method')
    axes[0].set_ylabel('Score')
    axes[0].set_title('Baseline Detector Performance Comparison')
    axes[0].set_xticks(x + width * 1.5)
    axes[0].set_xticklabels(results_df['method'])
    axes[0].legend()
    axes[0].grid(True, alpha=0.3, axis='y')
    
    # Speed comparison
    axes[1].bar(results_df['method'], results_df['train_time'], alpha=0.7, label='Train Time')
    axes[1].bar(results_df['method'], results_df['predict_time'], alpha=0.7, label='Predict Time')
    axes[1].set_xlabel('Detection Method')
    axes[1].set_ylabel('Time (seconds)')
    axes[1].set_title('Speed Comparison')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Baseline results not found. Run quick_test_baseline.py first.")

## 7. Key Findings & Insights

### Dataset Characteristics:
- **83 sensor features** across 4 industrial processes
- **Process P1** has the most sensors (38), indicating primary process control
- No missing values in the loaded samples
- Attack ratio in test data: ~2.7%

### Baseline Performance:
- **Isolation Forest** achieved best accuracy (82.77%)
- **Z-Score** achieved highest recall (89.63%) - catches most attacks
- All methods have low precision (high false positives)
- This is expected for simple baseline methods

### Next Steps:
1. **Feature Engineering** (Week 2)
   - Create statistical features (mean, std, rolling windows)
   - Extract temporal features
   - Use correlation insights

2. **ML Models** (Week 2)
   - Random Forest with engineered features
   - XGBoost for better performance
   - Target: >90% accuracy, <10% FPR

3. **Deep Learning** (Week 3)
   - LSTM for temporal patterns
   - Autoencoder for anomaly detection
   - Target: >93% accuracy, <5% FPR

## 8. Save Exploration Results

In [ ]:
# Save key insights
insights = {
    'dataset': 'HAI-21.03',
    'train_samples': len(train_df),
    'test_samples': len(test_df),
    'num_features': len(sensor_cols),
    'attack_ratio': test_info.get('attack_ratio', 0),
    'best_baseline_method': 'isolation_forest',
    'best_accuracy': 0.8277,
    'phase': 'Phase 3 - Baseline Testing',
    'date': '2025-11-06'
}

insights_df = pd.DataFrame([insights])
insights_file = project_root / 'results' / 'metrics' / 'phase3_insights.csv'
insights_df.to_csv(insights_file, index=False)

print(f"✅ Insights saved to: {insights_file}")
print("\n" + "="*70)
print(" 📊 PHASE 3: DATA EXPLORATION COMPLETED!")
print("="*70)
print("\nNext: Start Phase 4 - Feature Engineering")